## Reference
[huggingface document](https://huggingface.co/docs)<br>
[huggingface notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/language_modeling.ipynb)<br>
[huggingface transformer tokenizer](https://huggingface.co/docs/transformers/v4.29.0/en/main_classes/tokenizer#tokenizer)<br>
[ckip transformer(繁體中文tokenizer和model)](https://github.com/ckiplab/ckip-transformers)<br>
[ckip document](https://ckip-transformers.readthedocs.io/en/latest/_api/ckip_transformers.nlp.driver.html#ckip_transformers.nlp.driver.CkipWordSegmenter)

In [1]:
from pathlib import Path
from functools import partial
import re
import numpy as np
import pandas as pd
import torch
import scipy

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'


from pandarallel import pandarallel
# Adjust the number of workers if you want
pandarallel.initialize(progress_bar=True, verbose=0) # use all workers if nb_workers does not set

import datasets # huggingface dataset
from transformers import BertTokenizerFast, AutoModel


# tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese', tokenizer_kwargs={'use_ckip': True})

from tqdm import tqdm
tqdm.pandas() # for progress_apply

from hw3_utils import (
    load_json,
    jsonl_dir_to_df,
    calculate_precision,
    calculate_recall,
)

2023-05-11 11:53:15.605040: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 11:53:16.273972: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64
2023-05-11 11:53:16.274054: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64
2023-05-11 11:53:16.274059: W tensorflow/compiler/tf2tensorrt/utils/py_utils

In [2]:
def tokenize(input_dataset: datasets.Dataset) -> datasets.Dataset:
    """This function performs Chinese word segmentation and removes stopwords.

    Args:
        input_dataset (datasets.Dataset): entire dataset

    Returns:
        datasets.Dataset with 2 columns:
            attention_mask
            input_ids: tokenized and encoded text
    """


    return tokenizer(input_dataset["text"], truncation=True, max_length=1024)


In [3]:
wiki_path = "data/wiki-pages"
wiki_cache = "wiki"
target_column = "text"

wiki_cache_path = Path(f"data/{wiki_cache}.pkl")
if wiki_cache_path.exists():
    wiki_pages = pd.read_pickle(wiki_cache_path)
else:
    # You need to download `wiki-pages.zip` from the AICUP website
    wiki_pages = jsonl_dir_to_df(wiki_path)
    # wiki_pages are combined into one dataframe, so we need to reset the index
    wiki_pages = wiki_pages.reset_index(drop=True)

    # save the result to a pickle file
    wiki_pages.to_pickle(wiki_cache_path, protocol=4)

tokenized_path = Path(f"data/tokenized")
if tokenized_path.exists():
    tokenized_datasets = datasets.load_from_disk(tokenized_path)
else:

    wiki_dataset = datasets.Dataset.from_pandas(wiki_pages)
    
    tokenized_datasets = wiki_dataset.map(tokenize, batched=True, num_proc=8, remove_columns=["id", "text", "lines"])
    # save the result to a pickle file
    tokenized_datasets.save_to_disk(tokenized_path)

In [4]:
# block_size = tokenizer.model_max_length
block_size = 1024
print("block size: {}".format(block_size))

block size: 1024


In [5]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [6]:
concated_path = "data/concated"
if Path(concated_path).exists():
    lm_datasets = datasets.load_from_disk(concated_path)
else:
    lm_datasets = tokenized_datasets.map(
        group_texts,
        batched=True,
        batch_size=1000,
        num_proc=4,
    )
    lm_datasets.save_to_disk(concated_path)

In [7]:
# 比較 encode 再 decode 和原文的差異
text_index = 5
print("原始文字：{}".format(wiki_pages["text"][text_index]))
print("處理過後：{}".format(tokenizer.decode(lm_datasets[0]["input_ids"]).split("[SEP]")[text_index]))

原始文字：澳門體育如同其他地方一樣 ， 是當地的重要文化之一 ， 澳門市民比較經常參與的體育活動有足球 、 籃球 、 排球 、 慢跑 、 游泳 、 乒乓球 、 網球和羽毛球 。 澳門每年都殷行一些地區的舉辦 ， 但是由於澳門人口比較少 （ 接近70萬人 ） ， 職業聯賽的舉行在財政上不可行 ， 因此 ， 大多數參賽者都只是業餘運動愛好者 。 有很多體育協會和俱樂部經常在澳門組織當地或者區域性的體育活動和競賽 。 由於帶有業餘性質 ， 大部分體育協會和俱樂部都能夠獲得澳門政府的補貼或贊助 。 澳門特別行政區政府體育局專門負責澳門的體育發展 ， 以及重大國際體育賽事在澳門的舉辦 （ 如澳門格蘭披治大賽車 ） 。
處理過後： [CLS] 澳 門 體 育 如 同 其 他 地 方 一 樣 ， 是 當 地 的 重 要 文 化 之 一 ， 澳 門 市 民 比 較 經 常 參 與 的 體 育 活 動 有 足 球 、 籃 球 、 排 球 、 慢 跑 、 游 泳 、 乒 乓 球 、 網 球 和 羽 毛 球 。 澳 門 每 年 都 殷 行 一 些 地 區 的 舉 辦 ， 但 是 由 於 澳 門 人 口 比 較 少 （ 接 近 70 萬 人 ） ， 職 業 聯 賽 的 舉 行 在 財 政 上 不 可 行 ， 因 此 ， 大 多 數 參 賽 者 都 只 是 業 餘 運 動 愛 好 者 。 有 很 多 體 育 協 會 和 俱 樂 部 經 常 在 澳 門 組 織 當 地 或 者 區 域 性 的 體 育 活 動 和 競 賽 。 由 於 帶 有 業 餘 性 質 ， 大 部 分 體 育 協 會 和 俱 樂 部 都 能 夠 獲 得 澳 門 政 府 的 補 貼 或 贊 助 。 澳 門 特 別 行 政 區 政 府 體 育 局 專 門 負 責 澳 門 的 體 育 發 展 ， 以 及 重 大 國 際 體 育 賽 事 在 澳 門 的 舉 辦 （ 如 澳 門 格 蘭 披 治 大 賽 車 ） 。 


In [ ]:
model = AutoModel.from_pretrained('ckiplab/bert-base-chinese')

In [ ]:
with torch.no_grad():
    # encoded_layers, _ = model(tokens_tensor, token_type_ids=segments_tensors)